In [1]:
import sys
import cv2
import numpy as np
import os
import time
import subprocess

In [2]:
def find_webcam_index(device_name):
    command = "v4l2-ctl --list-devices"
    output = subprocess.check_output(command, shell=True, text=True)
    devices = output.split('\n\n')

    for device in devices:
        #print(device)
        if device_name in device:
            lines = device.split('\n')
            for line in lines:
                if "video" in line:
                    parts = line.split()
                    for part in parts:
                        if part.startswith('/dev/video'):
                            print(part)
                            return (part[10:])



In [16]:
import cv2
import numpy as np
from ultralytics import YOLO
import sys
# Initialize the YOLOv8 model
model = YOLO('/home/jonny/projects/xARM_rubiks_cube/from_roboflow/runs/detect/train10/weights/best.pt')#'yolov8s.pt')  # Make sure to provide the correct path to your YOLOv8 model file
cam_i = int(find_webcam_index("C922 Pro Stream Webcam"))
# Open a connection to the webcam (0 is the default camera)
cap = cv2.VideoCapture(cam_i)

if not cap.isOpened():
    print("Error: Could not open video device.")
    sys.exit(1)

# Set camera properties (optional)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
st_vid = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
vid_out = cv2.VideoWriter('./vid_out.mp4', st_vid, 4, (640,480))
start_time = time.time()
print(start_time)
close_vid_file = True

while True:
    strt = time.time()
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform inference on the frame
    results = model(frame, verbose=False)

    # Process the results
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Extract box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = int(box.cls[0])

            # Draw the bounding box and label on the frame
            label = f"{model.names[cls]} {conf:.2f}"
            if conf > 0.50:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            # cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('YOLOv8 Detection', frame)
    
    if (time.time() - start_time > 8):
        if (time.time()- start_time < 12):
            vid_out.write(frame)
        elif (time.time() -start_time >= 12) and close_vid_file:
            vid_out.release()
            print(time.time())
            close_vid_file = False



    # print(1/(time.time() - strt))
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close windows
cap.release()
cv2.destroyAllWindows()

/dev/video2
1721989901.0232382
1721989913.0695865


In [24]:
from torchinfo import summary
import torch
model = YOLO('/home/jonny/projects/xARM_rubiks_cube/from_roboflow/runs/detect/train10/weights/best.pt')#'yolov8s.pt')  # Make sure to provide the correct path to your YOLOv8 model file

# dummy_input = torch.randn(1, 3, 640, 480).to(next(model.parameters()).device)

summary(model.model, input_size=(1,3,640,480))

Layer (type:depth-idx)                             Output Shape              Param #
DetectionModel                                     [1, 10, 6300]             --
├─Sequential: 1-1                                  --                        --
│    └─Conv: 2-1                                   [1, 48, 320, 240]         --
│    │    └─Conv2d: 3-1                            [1, 48, 320, 240]         (1,296)
│    │    └─BatchNorm2d: 3-2                       [1, 48, 320, 240]         (96)
│    └─Detect: 2-136                               --                        (recursive)
│    │    └─ModuleList: 3-158                      --                        (recursive)
│    └─Conv: 2-3                                   [1, 96, 160, 120]         --
│    │    └─Conv2d: 3-4                            [1, 96, 160, 120]         (41,472)
│    │    └─BatchNorm2d: 3-5                       [1, 96, 160, 120]         (192)
│    └─Detect: 2-136                               --                        (rec